Automatic Differentiation with ``torch.autograd``
=======================================

ニューラルネットワークを訓練する際、その学習アルゴリズムとして、基本的には**バックプロパゲーション（back propagation）**が使われる．

バックプロパゲーションでは、モデルの重みなどの各パラメータは、損失関数に対するその変数の微分値（勾配）に応じて調整されるので各パラメータの勾配が重要な役割を持つ．

一見すると，各パラメータの微分値を求めるのは非常に煩雑で面倒な感じがするが，Pytorchでは``torch.autograd`` という微分エンジンが組み込まれていて計算グラフに対する勾配の自動計算を支援してくれるのですごい便利．




簡単な計算グラフを用意するのでここで``torch.autograd`` を確認していく．

In [ ]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

テンソル、関数、計算グラフの関係
------------------------------------------

上記のコードは以下の**計算グラフ(computational graph)**を示していル．

<img src="https://pytorch.org/tutorials/_images/comp-graph.png" width=50% alt="各パラメータ" title="損失関数の図">

この計算グラフでは``w``と``b``が最適化したいパラメータになる．

コードからも確認できるように，requires_gradという引数をTrueにすることで勾配計算を許可する形になり，``backward``実行後に各パラメータの``.grad``に保存されることになる．


In [ ]:
loss.backward()

In [ ]:
print("wの勾配",w.grad)

print("bの勾配",b.grad)

wの勾配 tensor([[0.0532, 0.3261, 0.0416],
        [0.0532, 0.3261, 0.0416],
        [0.0532, 0.3261, 0.0416],
        [0.0532, 0.3261, 0.0416],
        [0.0532, 0.3261, 0.0416]])
bの勾配 tensor([0.0532, 0.3261, 0.0416])


これは次の計算を求めていることに他ならない

$\frac{\partial loss}{\partial w}$ 、$\frac{\partial loss}{\partial b}$ 

``requires_grad``はテンソルを定義する際、もしくはその後に、``x.requires_grad_(True)``を実行するなどして指定することもできる．

注意：無闇に勾配追跡をONにしてしまうとコンピュータが裏で保持している計算グラフが莫大なものになってしまい計算の遅延に繋がりかねないので必要な時だけ``requires_grad``をTrueにする．

``backward``後にもう一度勾配計算を行うことは仕様上できない．

どうしてももう一度行いたい場合は，``backward``の際に引数に``retain_graph=True``を渡すと良い．

これを行うことで損失関数のヘッセ行列を得ることができる．

勾配計算をしない方法
---------------------------

デフォルトでは、``requires_grad=True``である全てのテンソルは計算履歴が保持され、勾配計算可能な状態になるが前述したように計算グラフは必要がないときには作らないほうが良い．

''torch.no_grad''を使って計算グラフが不要なコードはまとめるのが普通．

In [ ]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


また，``@torch.no_grad()``でラップすることで関数全体を包むこともできる．

訓練済みのモデルを使って予測を行う際などには勾配計算を行う必要はないため全体をこれでラップすることも多い．

勾配の計算、追跡を不能にしたいケースの紹介

- ネットワークの一部のパラメータを固定したい（frozen parameters）ケース。これは[ファインチューニング](https://colab.research.google.com/github/YutaroOgawa/pytorch_tutorials_jp/blob/main/notebook/2_Image_Video/2_1_transfer_learning_tutorial_jp.ipynb)時によくあるケース．



- 順伝搬の計算スピードを高速化したいケース．
